## Boulder detection methodology

- Author: Tim Nagle-McNaughton

- Contact: timnaglemcnaughton@unm.edu

- Requires Python 3+, 64-bit recommended

- This code is overly verbose (hopefully) for clarity

## 1. Data production

### 1.1 User input

### These values should be edited by the user:

In [ ]:
# any default workspace. Can be a folder or a geodatabase
workspace = 
# the raster you want to process
rast_to_process =     
# where your data will be saved
output_folder =        
# the minimum pixel iteration value (inclusive)
rast_min = 
# the maximum pixel threshold value (not inclusive)
rast_max = 
# the inverval between threshold values (e.g. min = 0, max = 100, step = 10 -> 10 files: 0, 10, 20, etc.)
rast_step = 
# the maximum value for the 'dark' brightness interval 
dark_interval_max = 
# the maxmimum value for the 'medium interval'
medium_interval_max = 

### 1.2 Import python modules and make subfolders for output

In [ ]:
### settings and setup ###
# imports
import arcpy
import os
import shutil
from arcpy.sa import *
from arcpy import env
import glob

# ArcGIS setup
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

# make a directory for the junk files
if not os.path.exists(os.path.join(output_folder, "junk\\")):
    os.makedirs(os.path.join(output_folder, "junk\\"))  
# set junk output strings
junk_path = os.path.join(output_folder, "junk\\")

# make a directory for the selected threshold files
if not os.path.exists(os.path.join(output_folder, "threshold\\")):
    os.makedirs(os.path.join(output_folder, "threshold\\"))
# set the threshold strings
thresh_path = os.path.join(output_folder, "threshold\\")

# make a directory for the final files
if not os.path.exists(os.path.join(output_folder, "final\\")):
    os.makedirs(os.path.join(output_folder, "final\\"))
# set the final strings
final_path = os.path.join(output_folder, "final/")

print("Setup complete... \n")

### 1.3 Apply the average filter with a 20-pixel radius

- Files are saved to ~ / junk / \*\_20\_rad\_average.tif

In [ ]:
# apply mean filter
neighborhood = NbrCircle(20)
avg20_rast = arcpy.sa.FocalStatistics(rast_to_process, neighborhood, "MEAN",  "")

# save the output to the junk folder
avg20_out = junk_name +  "_20_rad_average.tif"
avg20_rast.save(avg20_out)

print("20 pixel average complete...\n")

### 1.4 Make a mask to split the raster into the three (dark, medium, light) brightness intervals

- Files are saved to ~ / junk / \*\_mask.tif

In [ ]:
# generate the masks
dark_mask = arcpy.Raster(avg20_out) < (dark_interval_max + 1)
medium_mask = (arcpy.Raster(avg20_out) > dark_interval_max) & (arcpy.Raster(avg20_out) < (medium_interval_max + 1))
light_mask = arcpy.Raster(avg20_out) > medium_interval_max

#save the output to the intermediate folder
dark_mask_out = junk_name + "_dark_mask.tif"
medium_mask_out = junk_name + "_medium_mask.tif"
light_mask_out = junk_name + "_light_mask.tif"

dark_mask.save(dark_mask_out)
medium_mask.save(medium_mask_out)
light_mask.save(light_mask_out)

print("Dark, medium, and light masks generated...\n")

### 1.5 Split the raster into three brightness intervals

- Files are saved to ~ / junk / \*\_zero.tif

In [ ]:
# generate the brightness rasters
dark = arcpy.Raster(dark_mask_out) * arcpy.Raster(rast_to_process)
medium = arcpy.Raster(medium_mask_out) * arcpy.Raster(rast_to_process)
light = arcpy.Raster(light_mask_out) * arcpy.Raster(rast_to_process)

# save the output to the intermediate folder
dark_out = junk_name + "_dark_zero.tif"
medium_out = junk_name + "_medium_zero.tif"
light_out = junk_name + "_light_zero.tif"
dark.save(dark_out)
medium.save(medium_out)
light.save(light_out)

print("Dark, medium, and light rasters generated...\n")

### 1.6 Set 0 values to 'Null'

- Files are saved to ~ / junk / \*\_null.tif

In [ ]:
# generate the null values in the rasters
dark_mask_null = SetNull(dark_out, dark_out, "VALUE < 1")
medium_mask_null = SetNull(medium_out, medium_out, "VALUE < 1")
light_mask_null = SetNull(light_out, light_out, "VALUE < 1")

# save the output
dark_mask_null_out = junk_path + "null_dark.tif"
medium_mask_null_out = junk_path + "null_medium.tif"
light_mask_null_out = junk_path + "null_light.tif"
dark_mask_null.save(dark_mask_null_out)
medium_mask_null.save(medium_mask_null_out)
light_mask_null.save(light_mask_null_out)

print("Final dark, medium, and light rasters generated...\n")

### 1.7 Apply the 2x2 range filter, generate threshold values

- Range files are saved to ~ / junk / \*\_2x2\_range.tif
 
- Threshold files are saved to  ~ / threshold / <threshold\_value>.tif

In [ ]:
# apply a 2x2 range filter
interval_list = [dark_mask_null_out, medium_mask_null_out, light_mask_null_out]
for interval in interval_list:
    neighborhood = NbrRectangle(2)
    rast_range = arcpy.sa.FocalStatistics(interval, neighborhood, "RANGE",  "")
    
    # make a better basename
    basename = arcpy.Describe(interval).basename[5:]
    
    # save the output #
    range_out = junk_path + basename + "_2x2_range.tif"
    rast_range.save(range_out)

    ### peform raster calculation ###
    # iteratively select 'boulder' pixels above the threshold set above
    for value in range(rast_min, rast_max, rast_step):
        calc_above = arcpy.Raster(range_out) > value
        
        # save the output
        calc_out = thresh_path + basename + "_"+ str(value) + ".tif"
        calc_above.save(calc_out)

print("Threshold pixel values for generated...\n")

arcpy.CheckInExtension("Spatial");

## 2. Data conversion
**At this point, the user should examine the interval rasters (stored in ~/threshold/) in ArcGIS to determine two values:**

1. **Conservative:** The pixel threshold at which there are no false-positive boulder pixels (no dune crests have boulder pixels). This value will be larger than the second value.
2. **Liberal:** The pixel threshold at which there are no false-negative boulder pixels (every boulder has a pixel). This value will be smaller than the first.

Additionally, the user must select the type of minimum bounding geometry they want to use:
- RECTANGLE_BY_AREA — The rectangle of the smallest area enclosing an input feature.
- RECTANGLE_BY_WIDTH — The rectangle of the smallest width enclosing an input feature.
- CONVEX_HULL — The smallest convex polygon enclosing an input feature.
- CIRCLE — The smallest circle enclosing an input feature.
- ENVELOPE — The envelope of an input feature.

### 2.1 User input

### These values should be edited by the user:

In [ ]:
# get variables 
dark_upper_value = # Conservative
dark_lower_value = # Liberal

medium_upper_value = # Conservative
medium_lower_value = # Liberal

light_upper_value = # Conservative
light_lower_value = # Liberal

# select the type of minimum bounding geometry
# RECTANGLE_BY_AREA, RECTANGLE_BY_WIDTH, CONVEX_HULL, CIRCLE, ENVELOPE
user_selected_MBG = 

### 2.2 Convert the rasters to simplified polygons
 
- Files are saved to ~ / junk / \*\_s.tif

In [ ]:
# generate list of the dark rasters
arcpy.env.workspace = thresh_path
dark_raster_list = arcpy.ListRasters("*dark*" + str(dark_upper_value) + "*")
dark_raster_list.extend(arcpy.ListRasters("*dark*" + str(dark_lower_value) + "*"))

# convert to polygons
for dark in dark_raster_list:
    # make a better name
    raster_name = junk_path + arcpy.Describe(dark).baseName + "_s.shp"
    # convert
    arcpy.conversion.RasterToPolygon(dark, raster_name, "SIMPLIFY")
    print(arcpy.Describe(raster_name).basename + " has been processed...")
    
# generate list of the medium rasters
arcpy.env.workspace = thresh_path
medium_raster_list = arcpy.ListRasters("*medium*" + str(medium_upper_value) + "*")
medium_raster_list.extend(arcpy.ListRasters("*medium*" + str(medium_lower_value) + "*"))

# convert to polygons
for med in medium_raster_list:
    # make a better name
    raster_name = junk_path + arcpy.Describe(med).baseName + "_s.shp"
    # convert
    arcpy.conversion.RasterToPolygon(med, raster_name, "SIMPLIFY")
    print(arcpy.Describe(raster_name).basename + " has been processed...")
    
# generate list of the light rasters
arcpy.env.workspace = thresh_path
light_raster_list = arcpy.ListRasters("*light*" + str(light_upper_value) + "*")
light_raster_list.extend(arcpy.ListRasters("*light*" + str(light_lower_value) + "*"))

# convert to polygons
for light in light_raster_list:
    # make a better name
    raster_name = junk_path + arcpy.Describe(light).baseName + "_s.shp"
    # convert
    arcpy.conversion.RasterToPolygon(light, raster_name, "SIMPLIFY")
    print(arcpy.Describe(raster_name).basename + " has been processed...")

### 2.3 Select only the polygons that represent boulders (i.e. not the spaces inbetween)
 
- Files are saved to ~ / junk / \*\_1.tif

In [ ]:
# generate list of polygons
arcpy.env.workspace = junk_path
process_list = arcpy.ListFeatureClasses("*_s.shp")

# select only 1 values
for feature in process_list:
    # make a better name
    feature_name = junk_path + arcpy.Describe(feature).basename + "_1.shp"
    # SQL query
    SQL = '"gridcode" = 1'
    # select only 1 values
    arcpy.analysis.Select(feature, feature_name, SQL)

print("Selection of boulder values completed...")

### 2.4 Generate the minimum bounding geometry

- Files are saved to ~ / final / \*\_bd.tif

In [ ]:
#generate another list of polygons
arcpy.env.workspace = junk_path
one_list = arcpy.ListFeatureClasses("*_1.shp")

# generate minimum bounding geometry
for polygon in one_list:
    # make a better name
    poly_name = final_path + arcpy.Describe(polygon).baseName[:-4] + "_bd.shp"
    # add an area field
    arcpy.management.AddField(polygon, "Area_orig", "FLOAT")
    # calculate area
    arcpy.management.CalculateField(polygon, "Area_orig", "!shape.area@squaremeters!", "PYTHON_9.3")
    # generate bounding geometry
    arcpy.management.MinimumBoundingGeometry(polygon, poly_name, str(user_selected_MBG), "", "", "MBG_FIELDS")

print("Bounding geometry completed...")

### 2.5 Add minimum bounding area values

In [ ]:
# generate a list of MBG polygons
arcpy.env.workspace = final_path
bound_list = arcpy.ListFeatureClasses("*_bd.shp")

# caluculate the MBG area
for bound in bound_list:
    # add an area field
    arcpy.management.AddField(bound, "MBG_Area", "FLOAT")
    # calculate area
    arcpy.management.CalculateField(bound, "MBG_Area", "!shape.area@squaremeters!", "PYTHON_9.3")

print("Area added...")

### 2.6 Convert polygons to points (Optional)

- Files are saved to ~ / final / \*\_pnt.tif

In [ ]:
# generate a list of the final files
arcpy.env.workspace = final_path
feat_list = arcpy.ListFeatureClasses("*_bd.shp")

# convert all the features
for feature in feat_list:
    #make a better name 
    point_name = arcpy.Describe(feature).basename + "_pnt.shp"
    arcpy.management.FeatureToPoint(feature, point_name)

print("Points generated...")

### 2.7 Add the X and Y coordinates to the points (Optional)

In [ ]:
### add lat long ###
# generate a list of the final files
arcpy.env.workspace = final_path
file_list = arcpy.ListFeatureClasses("*_pnt.shp")

# add XY points
for points in file_list:
    arcpy.management.AddXY(points)

print("Coordinates added...")

### 2.8 Convert all the files to spreadsheets (Optional)

- Files are saved to ~ / final /

In [ ]:
# generate a list of the final files
arcpy.env.workspace = final_path
file_list = arcpy.ListFeatureClasses()

# convert the tables to CSVs
for final_file in file_list:
    # set output 
    table_out = arcpy.Describe(final_file).baseName + ".csv"
    #make a table view
    table_view = arcpy.Describe(final_file).baseName + "_table"
    arcpy.management.MakeTableView(final_file, table_view)
    # export the table
    arcpy.conversion.TableToTable(table_view, final_path, table_out)
    
print("Final tables generated...")